# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [1]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')
print(customer.head())
print(uselog_months.head())

  customer_id   name class gender  start_date end_date campaign_id  \
0    OA832399   XXXX   C01      F  2015-05-01      NaN         CA1   
1    PL270116  XXXXX   C01      M  2015-05-01      NaN         CA1   
2    OA974876  XXXXX   C01      M  2015-05-01      NaN         CA1   
3    HD024127  XXXXX   C01      F  2015-05-01      NaN         CA1   
4    HD661448  XXXXX   C03      F  2015-05-01      NaN         CA1   

   is_deleted class_name  price campaign_name      mean  median  max  min  \
0           0     オールタイム  10500            通常  4.833333     5.0    8    2   
1           0     オールタイム  10500            通常  5.083333     5.0    7    3   
2           0     オールタイム  10500            通常  4.583333     5.0    6    3   
3           0     オールタイム  10500            通常  4.833333     4.5    7    2   
4           0        ナイト   6000            通常  3.916667     4.0    6    1   

   routine_flg   calc_date  membership_period  
0            1  2019-04-30                 47  
1            1  2019

In [5]:
year_months = list(uselog_months['年月'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月']==year_months[i]]
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月']==year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/Cellar/jupyterlab/2.2.8/libexec/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

### ノック43：継続顧客のデータを作成しよう

### ノック44：予測する月の在籍期間を作成しよう

### ノック45：欠損値を除去しよう

### ノック46：文字列型の変数を処理できるように整形しよう

### ノック47：決定木を用いて退会予測モデルを作成してみよう

### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

### ノック49：モデルに寄与している変数を確認しよう

### ノック50：顧客の退会を予測しよう